we have like a dataset in the folder stocks/train and also in stocks/test for a market stock price prediction task.

create a full LSTM to forcast non linear time series dataset when tensorflow keras and u will found the data in the stocks file there is a train folder and test folder
and this an overview about our data

In [23]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import os
import glob

In [24]:
# Path to the training data folder
train_path = "stocks/train/"

# Get a list of all CSV files in the training folder
all_train_files = glob.glob(os.path.join(train_path, "*.csv"))

# Read each CSV file and concatenate them into a single DataFrame
df_list = [pd.read_csv(file) for file in all_train_files]
train_df = pd.concat(df_list, ignore_index=True)

# Sort by date (optional but good practice)
train_df = train_df.sort_values(by='Date').reset_index(drop=True)

print(f"Combined {len(all_train_files)} training files into one DataFrame with {len(train_df)} rows.")
train_df.head()

Combined 354 training files into one DataFrame with 1404614 rows.


,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0
1,1962-01-03,6.532155,6.632280,6.524145,6.632280,1.560212,74500.0
2,1962-01-04,6.632280,6.664320,6.632280,6.632280,1.560212,80500.0
3,1962-01-05,6.632280,6.656310,6.616260,6.624270,1.558326,70500.0
4,1962-01-08,6.608250,6.608250,6.339915,6.408000,1.507450,93800.0


In [25]:
# Create a new dataframe with only the 'Close' column
data = train_df.filter(['Close'])

# Convert the dataframe to a numpy array
dataset = data.values

# Scale the data. It's important to fit the scaler ONLY on the training data.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# --- Create the training data set from multiple files ---
# We create sequences for each file separately to prevent sequences from overlapping between different stocks
x_train = []
y_train = []

for file in all_train_files:
    df = pd.read_csv(file)
    close_prices = df.filter(['Close']).values
    # Use the same scaler that was fit on the combined training data
    scaled_prices = scaler.transform(close_prices) 

    for i in range(60, len(scaled_prices)):
        x_train.append(scaled_prices[i-60:i, 0])
        y_train.append(scaled_prices[i, 0])

# Convert the x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Reshape the data for the LSTM model [samples, time_steps, features]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print(f"Created {x_train.shape[0]} training sequences.")

Created 1383479 training sequences.


In [27]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print a summary of the model architecture
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 60, 50)         │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 60, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 60, 1)          │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
43234/43234 ━━━━━━━━━━━━━━━━━━━━ 236s 5ms/step - loss: nan
Epoch 2/10
43234/43234 ━━━━━━━━━━━━━━━━━━━━ 220s 5ms/step - loss: nan
Epoch 3/10
43234/43234 ━━━━━━━━━━━━━━━━━━━━ 224s 5ms/step - loss: nan
Epoch 4/10
43234/43234 ━━━━━━━━━━━━━━━━━━━━ 236s 5ms/step - loss: nan
Epoch 5/10
43234/43234 ━━━━━━━━━━━━━━━━━━━━ 240s 6ms/step - loss: nan
Epoch 6/10
23502/43234 ━━━━━━━━━━━━━━━━━━━━ 1:41 5ms/step - loss: nan

In [1]:
# Path to the testing data folder
test_path = "stocks/test/"

# Get a list of all CSV files in the testing folder
all_test_files = glob.glob(os.path.join(test_path, "*.csv"))

# Combine all test files
df_list_test = [pd.read_csv(file) for file in all_test_files]
test_df = pd.concat(df_list_test, ignore_index=True)
test_df = test_df.sort_values(by='Date').reset_index(drop=True)

# Get the actual closing prices for later plotting
actual_prices = test_df.filter(['Close']).values

# Concatenate the training and test data to create the input sequences for testing
total_dataset = pd.concat((train_df['Close'], test_df['Close']), axis=0)

# Get the inputs for the test data
model_inputs = total_dataset[len(total_dataset) - len(test_df) - 60:].values
model_inputs = model_inputs.reshape(-1, 1)
model_inputs = scaler.transform(model_inputs) # Use the same scaler

# Create the test data set
x_test = []
for i in range(60, len(model_inputs)):
    x_test.append(model_inputs[i-60:i, 0])

# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

print(f"Created {x_test.shape[0]} testing sequences.")

NameError: name 'glob' is not defined

In [ ]:
# Get the models predicted price values
predicted_prices = model.predict(x_test)
# Inverse transform the predictions to get the actual price values
predicted_prices = scaler.inverse_transform(predicted_prices)

In [ ]:
# Plot the data
plt.figure(figsize=(16,8))
plt.title('Stock Price Prediction on All Test Data')
plt.xlabel('Time (in order of the combined test data)')
plt.ylabel('Close Price USD ($)')
plt.plot(actual_prices, color='blue', label='Actual Prices')
plt.plot(predicted_prices, color='red', label='Predicted Prices')
plt.legend()
plt.show()